In [1]:
import redis
import json

In [2]:
r = redis.Redis(host='redis-12442.c90.us-east-1-3.ec2.cloud.redislabs.com',port=12442,password='RHFgwK2pQ2ibFUqw8zrQBHRzK29lQtDw')
keys=r.keys('*') #getting the keys of each convesation
print(keys)

[b'tracker:49c872c0e39c4efca64d78faf40f0475', b'tracker:a8dcbf90b47f4767a554e41f8e5483d7', b'tracker:8d72b69da1f34ee8b0707dd0b1a87573', b'tracker:446832a509784fbc9ff67dd2547384c3', b'tracker:b1c3a26a4f17431f8579fa0156738456']


In [3]:
trackerdata=[]
for i in keys: 
    temp=r[i]
    temp = temp.decode("utf-8")
    trackerdata.append(json.loads(temp)) #each conversation is saved with its events as a list and trackername
conversations=[]
for i in trackerdata:
    conversations.append(i["events"]) #extracting only the events list of each conversation

In [5]:
for i in conversations[0]:
    print(i)

{'event': 'action', 'timestamp': 1686898029.3402743, 'metadata': {'model_id': 'dfaedd0a318e48e79c6d8c73abba0fe0', 'assistant_id': '20230421-154826-finite-valuation'}, 'name': 'action_session_start', 'policy': None, 'confidence': 1.0, 'action_text': None, 'hide_rule_turn': False}
{'event': 'session_started', 'timestamp': 1686898029.3422678, 'metadata': {'model_id': 'dfaedd0a318e48e79c6d8c73abba0fe0', 'assistant_id': '20230421-154826-finite-valuation'}}
{'event': 'action', 'timestamp': 1686898029.3422678, 'metadata': {'model_id': 'dfaedd0a318e48e79c6d8c73abba0fe0', 'assistant_id': '20230421-154826-finite-valuation'}, 'name': 'action_listen', 'policy': None, 'confidence': None, 'action_text': None, 'hide_rule_turn': False}
{'event': 'user', 'timestamp': 1686898030.242938, 'metadata': {'model_id': 'dfaedd0a318e48e79c6d8c73abba0fe0', 'assistant_id': '20230421-154826-finite-valuation'}, 'text': 'hi', 'parse_data': {'intent': {'name': 'greet', 'confidence': 0.9994363188743591}, 'entities': []

In [4]:
for conv in conversations:
    for i in conv:
        if i['event']=='user':
            print("user: ",i['text'])
            if len(i['parse_data']['entities'])>0:
                print('data entered: ',i['parse_data']['entities'][0]['entity'],"=",i['parse_data']['entities'][0]['value'])
        elif i['event']=="bot":
            print("bot: ",i["text"])
    print("\n")

user:  hi
bot:  Hey! How are you?
user:  item info
bot:  which of the details of the below you want :
user:  /price_details
bot:  Select Get item prices, to retrieve prices for all items at all locations.

Select Get specific price of item/loc, to retrieve the price of a specific item at a specific location.

Select Get price of item at all locs, to retrieve prices for a specific item at all available locations.

Please select a use case:
user:  /get_item_all_locs
bot:  enter item number please
user:  100150000
data entered:  loc = 100150000


user:  item info
bot:  which of the details of the below you want :
user:  /price_details
bot:  Select Get item prices, to retrieve prices for all items at all locations.

Select Get specific price of item/loc, to retrieve the price of a specific item at a specific location.

Select Get price of item at all locs, to retrieve prices for a specific item at all available locations.

Please select a use case:
user:  /get_specific_price
bot:  please g

In [10]:
intents=set()
intentdict=dict()
for conv in conversations:
    for i in conv:
        if i['event']=="user":
            if i['parse_data']['intent']['name'] not in intents:
                intents.add(i['parse_data']['intent']['name'])
                intentdict[i['parse_data']['intent']['name']]={i["text"]}
            else:
                intentdict[i['parse_data']['intent']['name']].add(i["text"])
for k,v in intentdict.items():
    print(k,":",v)




greet : {'hi', 'sup', 'hey'}
user_query : {'item info'}
price_details : {'price details', '/price_details'}
get_item_all_locs : {'/get_item_all_locs'}
location_info : {'100035001', '100150000'}
get_specific_price : {'/get_specific_price', 'particular item particular location'}
mood_unhappy : {'im happy', 'am sad'}
affirm : {'yes'}
deny : {'no'}
goodbye : {'bye'}
item_details : {'105', '100150000'}
location_details : {'location 105'}


https://www.kaggle.com/datasets/salmanfaroz/small-talk-intent-classification-data

In [10]:
import yaml

with open(r'D:\documents\internship\rasachatbot\data\nlu.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    nlu = yaml.load(file, Loader=yaml.FullLoader)
nlu=nlu["nlu"]
print(nlu)

[{'intent': 'greet', 'examples': '- hey\n- hello\n- hi\n- hello there\n- good morning\n- good evening\n- moin\n- hey there\n- hey dude\n- good morning\n- good evening\n- good afternoon\n- sup\n'}, {'intent': 'goodbye', 'examples': '- cu\n- good by\n- cee you later\n- good night\n- bye\n- goodbye\n- have a nice day\n- see you around\n- bye bye\n- see you later\n'}, {'intent': 'affirm', 'examples': '- yes\n- y\n- indeed\n- of course\n- that sounds good\n- correct\n'}, {'intent': 'deny', 'examples': "- no\n- n\n- never\n- I don't think so\n- don't like that\n- no way\n- not really\n"}, {'intent': 'mood_great', 'examples': '- perfect\n- great\n- amazing\n- feeling like a king\n- wonderful\n- I am happy\n- I am feeling very good\n- I am great\n- I am amazing\n- I am going to save the world\n- super stoked\n- extremely good\n- so so perfect\n- so good\n- so perfect\n'}, {'intent': 'mood_unhappy', 'examples': "- my day was horrible\n- I am sad\n- I don't feel very well\n- I am disappointed\n-

In [14]:
intentdict=dict()
for i in nlu:
    temp=i["examples"].split("\n- ")
    #temp[0]=temp[0][2:]
    #temp[-1]=temp[-1][-2]
    intentdict[i["intent"]]=temp
for k,v in intentdict.items():
    print(k,": ",v)
#print(intentdict)

greet :  ['- hey', 'hello', 'hi', 'hello there', 'good morning', 'good evening', 'moin', 'hey there', 'hey dude', 'good morning', 'good evening', 'good afternoon', 'sup\n']
goodbye :  ['- cu', 'good by', 'cee you later', 'good night', 'bye', 'goodbye', 'have a nice day', 'see you around', 'bye bye', 'see you later\n']
affirm :  ['- yes', 'y', 'indeed', 'of course', 'that sounds good', 'correct\n']
deny :  ['- no', 'n', 'never', "I don't think so", "don't like that", 'no way', 'not really\n']
mood_great :  ['- perfect', 'great', 'amazing', 'feeling like a king', 'wonderful', 'I am happy', 'I am feeling very good', 'I am great', 'I am amazing', 'I am going to save the world', 'super stoked', 'extremely good', 'so so perfect', 'so good', 'so perfect\n']
mood_unhappy :  ['- my day was horrible', 'I am sad', "I don't feel very well", 'I am disappointed', 'super sad', "I'm so sad", 'sad', 'very sad', 'unhappy', 'not good', 'not very good', 'extremly sad', 'so saad', 'so sad\n']
bot_challenge

In [18]:
intentdict=dict()
for i in nlu:
    temp=i["examples"].split("\n- ")
    temp[0]=temp[0][2:]
    temp[-1]=temp[-1][:-1]
    intentdict[i["intent"]]=set(temp)
for k,v in intentdict.items():
    print(k,": ",v)
#print(intentdict)

greet :  {'hello there', 'moin', 'good afternoon', 'hello', 'sup', 'hey there', 'hi', 'hey', 'good evening', 'hey dude', 'good morning'}
goodbye :  {'cu', 'bye', 'cee you later', 'good by', 'see you around', 'have a nice day', 'see you later', 'good night', 'goodbye', 'bye bye'}
affirm :  {'indeed', 'of course', 'correct', 'that sounds good', 'yes', 'y'}
deny :  {'never', "don't like that", 'not really', 'no way', "I don't think so", 'n', 'no'}
mood_great :  {'great', 'so good', 'so so perfect', 'I am going to save the world', 'I am feeling very good', 'extremely good', 'super stoked', 'I am happy', 'so perfect', 'I am amazing', 'amazing', 'perfect', 'feeling like a king', 'I am great', 'wonderful'}
mood_unhappy :  {'I am sad', 'very sad', 'not good', 'so saad', 'unhappy', 'not very good', 'my day was horrible', 'so sad', 'extremly sad', "I don't feel very well", 'sad', "I'm so sad", 'I am disappointed', 'super sad'}
bot_challenge :  {'am I talking to a human?', 'are you a bot?', 'am I